# get ocean source RH/SST and source ratio

In [ ]:
import numpy as np
import pandas as pd
import netCDF4
import math
import matplotlib.pyplot as plt
from netCDF4 import Dataset
import pyreadr
import xarray

In [ ]:
rdata = pyreadr.read_r("ERA3_data.RData") # import processed ERA5 datasets
month = ["01","02","03","04","05","06","07","08","09","10","11","12"]

In [ ]:
for mn in range(12):
    dataset = Dataset("utrack_climatology_1.0_"+month[mn]+".nc") # read UTrack dataset file by month
    lat = dataset.variables['sourcelat'][:].data
    lon = dataset.variables['sourcelon'][:].data
    moisture_flow = dataset.variables['moisture_flow'][:].data
    rhs_12mean_regrid = xarray.DataArray.to_numpy(rdata["rhs_12mean_regrid"])
    sst_12mean_regrid = xarray.DataArray.to_numpy(rdata["sst_12mean_regrid"])
    precip_12mean_regrid = xarray.DataArray.to_numpy(rdata["precip_12mean_regrid"])
    evap_12mean_regrid = xarray.DataArray.to_numpy(rdata["evap_12mean_regrid"])
    rhs_mean = np.zeros(shape=(360,180))
    sst_mean = np.zeros(shape=(360,180))
    operc = np.zeros(shape=(360,180))
    rhs = np.matrix.flatten(rhs_12mean_regrid[mn,:,:])
    sst = np.matrix.flatten(sst_12mean_regrid[mn,:,:])
    idr = np.where(~np.isnan(rhs))[0]
    for i in range(360):
        for j in range(180):
            flux = np.matrix.flatten(evap_12mean_regrid[mn,:,:]*np.exp(np.transpose(moisture_flow[:,:,j,i])*-0.1))
            rhs_mean[i,j] = sum(flux[idr]*rhs[idr])/sum(flux[idr])
            sst_mean[i,j] = sum(flux[idr]*sst[idr])/sum(flux[idr]) # Equation 4 in the main text
            operc[i,j] = sum(flux[idr])/sum(flux) # OMS ratio calculated in the main text
        print(mn,i)
    np.save("rhs_mean"+month[mn]+".npy",rhs_mean)
    np.save("sst_mean"+month[mn]+".npy",sst_mean)
    np.save("operc"+month[mn]+".npy",operc)

# get 1st cascade

In [ ]:
import numpy as np
import pandas as pd
import netCDF4
import math
import matplotlib.pyplot as plt
from netCDF4 import Dataset
import pyreadr
import xarray

In [ ]:
rdata = pyreadr.read_r("ERA3_data.RData") # import processed ERA5 datasets
month = ["01","02","03","04","05","06","07","08","09","10","11","12"]

In [ ]:
for mn in range(12):
    rhs_mean = np.load("rhs_mean"+month[mn]+".npy") # read direct OMS outputs
    sst_mean = np.load("sst_mean"+month[mn]+".npy")
    operc = np.load("operc"+month[mn]+".npy")
    dataset = Dataset("utrack_climatology_1.0_"+month[mn]+".nc") # read UTrack dataset file by month
    lat = dataset.variables['sourcelat'][:].data
    lon = dataset.variables['sourcelon'][:].data
    moisture_flow = dataset.variables['moisture_flow'][:].data
    rhs_12mean_regrid = xarray.DataArray.to_numpy(rdata["rhs_12mean_regrid"])
    sst_12mean_regrid = xarray.DataArray.to_numpy(rdata["sst_12mean_regrid"])
    precip_12mean_regrid = xarray.DataArray.to_numpy(rdata["precip_12mean_regrid"])
    evap_12mean_regrid = xarray.DataArray.to_numpy(rdata["evap_12mean_regrid"])
    rhs1c_mean = np.zeros(shape=(360,180))
    sst1c_mean = np.zeros(shape=(360,180))
    operc1c = np.zeros(shape=(360,180))
    rhs = np.matrix.flatten(rhs_12mean_regrid[mn,:,:])
    idc = np.where(np.isnan(rhs))[0]
    rhs = np.matrix.flatten(rhs_mean)
    sst = np.matrix.flatten(sst_mean)
    operc = np.matrix.flatten(operc)
    for i in range(360):
        for j in range(180):
            flux = np.matrix.flatten(evap_12mean_regrid[mn,:,:]*np.exp(np.transpose(moisture_flow[:,:,j,i])*-0.1))
            rhs1c_mean[i,j] = sum(flux[idc]*rhs[idc])/sum(flux[idc])
            sst1c_mean[i,j] = sum(flux[idc]*sst[idc])/sum(flux[idc]) # Equation S3 in the Supporting Information
            operc1c[i,j] = sum(flux[idc]*operc[idc])/sum(flux[idc]) # Equation S1 in the Supporting Information
        print(mn,i)
    np.save("rhs1c_mean"+month[mn]+".npy",rhs1c_mean)
    np.save("sst1c_mean"+month[mn]+".npy",sst1c_mean)
    np.save("operc1c"+month[mn]+".npy",operc1c)